In [13]:
from imutils.object_detection import non_max_suppression
import numpy as np
import pytesseract
import argparse
import cv2
%matplotlib inline

In [14]:
east = 'frozen_east_text_detection.pb'
min_confidence = 0.5
width = 320
height = 320
padding = 0.0

In [15]:
img = 'images/img_3.jpg'

In [16]:
def decode_predictions(scores, geometry):

	(numRows, numCols) = scores.shape[2:4]
	rects = []
	confidences = []

	for y in range(0, numRows):

		scoresData = scores[0, 0, y]
		xData0 = geometry[0, 0, y]
		xData1 = geometry[0, 1, y]
		xData2 = geometry[0, 2, y]
		xData3 = geometry[0, 3, y]
		anglesData = geometry[0, 4, y]

		for x in range(0, numCols):

			if scoresData[x] < min_confidence:
				continue

			(offsetX, offsetY) = (x * 4.0, y * 4.0)

			angle = anglesData[x]
			cos = np.cos(angle)
			sin = np.sin(angle)

			h = xData0[x] + xData2[x]
			w = xData1[x] + xData3[x]

			endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
			endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
			startX = int(endX - w)
			startY = int(endY - h)

			rects.append((startX, startY, endX, endY))
			confidences.append(scoresData[x])

	return (rects, confidences)


image = cv2.imread(img)
orig = image.copy()
(origH, origW) = image.shape[:2]

(newW, newH) = (width, height)
rW = origW / float(newW)
rH = origH / float(newH)

image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

layerNames = [
	"feature_fusion/Conv_7/Sigmoid",
	"feature_fusion/concat_3"]

# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(east)

blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
	(123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

(rects, confidences) = decode_predictions(scores, geometry)
boxes = non_max_suppression(np.array(rects), probs=confidences)

# initialize the list of results
results = []
output_text = []

def get_output_text(text):
	if text not in output_text:
		output_text.append(text)

# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:

	startX = int(startX * rW)
	startY = int(startY * rH)
	endX = int(endX * rW)
	endY = int(endY * rH)

	dX = int((endX - startX) * padding)
	dY = int((endY - startY) * padding)

	# apply padding to each side of the bounding box, respectively
	startX = max(0, startX - dX)
	startY = max(0, startY - dY)
	endX = min(origW, endX + (dX * 2))
	endY = min(origH, endY + (dY * 2))

	# extract the actual padded ROI
	roi = orig[startY:endY, startX:endX]

	config = ("-l eng --oem 1 --psm 7")
	text = pytesseract.image_to_string(roi, config=config)
	get_output_text(text)

	results.append(((startX, startY, endX, endY), text))

# sort the results bounding box coordinates from top to bottom
results = sorted(results, key=lambda r:r[0][1])
print("Results: ", output_text)



# Display founded Text On Image
def show_text_on_img(results):
	# loop over the results
	for ((startX, startY, endX, endY), text) in results:

		print("OCR TEXT")
		print("========")
		print("{}\n".format(text))

		text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
		output = orig.copy()
		cv2.rectangle(output, (startX, startY), (endX, endY),
			(0, 0, 255), 2)
		cv2.putText(output, text, (startX, startY - 20),
			cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

		# show the output image
		cv2.imshow("Text Detection", output)
		cv2.waitKey(0)

# Count ALL Detected Text from an Image
# def count_text(text):
# 	for t in text:
# 		count = text.count(t)
# 		print('{} = {}'.format(t, count))


[INFO] loading EAST text detector...
Results:  ['', 'BLUE', 'ia ise', 'a.', 'a ee', 'eae']


In [17]:
# show_text_on_img(results)
# count_text(output_text)

OCR TEXT


OCR TEXT
a.

OCR TEXT
ia ise

OCR TEXT
BLUE

OCR TEXT
BLUE

OCR TEXT
a ee

OCR TEXT
eae

 = 1
BLUE = 1
ia ise = 1
a. = 1
a ee = 1
eae = 1
